### Capstone Project

In [1]:
import pandas as pd
import numpy as np
import world_bank_data as wb

Reading data from the web:

We use the QoG OECD Time Series dataset to extract cross-national data for expenditure on environment protection, 
as percentage of total general government expenditure and world bank data of CO2 emission for corresponding countries
and years. We further use economic growth, final government consumption expenditure (% of GDP) and 
Gross capital formation (% of GDP) from World Bank data bank.

We first load the data from 1995.

The dataset can be found at : https://www.gu.se/en/quality-government/qog-data/data-downloads/oecd-dataset

In [9]:
data = pd.read_csv('http://www.qogdata.pol.gu.se/data/qog_oecd_ts_jan21.csv')
data = data[['cname', 'ccodealp', 'year', 'gfs_envr', 'wdi_co2', 'oecd_soxnox_t1a', 'oecd_soxnox_t1b', 'wdi_forest']]
data = data[data['year'] >= 1995]

Now we generate a missing values table for report for environment expenditure to weed out countries and periods with most missing values to get a nearly balanced panel dataset.

In [10]:
def gen_nan_report(data, x):
    countries = data['cname'].value_counts().index
    years = data['year'].value_counts().index
    missing_values = {}

    for i in range(len(countries)):
        country = countries[i]
        m1 = {}
        for j in range(len(years)):
            year = years[j]
            subset = data[np.logical_and(data['cname'] == country, data['year'] == year)][x]
            nans = subset.isnull().values.ravel().sum()
            m1.update({year : nans})
        missing_values.update({country : m1})
    return missing_values

countries = data['cname'].value_counts().index

nan_env = pd.DataFrame(gen_nan_report(data=data, x='gfs_envr'))

missing_value_per_country = {country : nan_env.loc[:, country].sum() for country in countries}

print(pd.Series(missing_value_per_country).sort_values())

Switzerland        1
Austria            1
Luxembourg         1
Germany            1
Italy              2
Sweden             2
Norway             2
Czech Republic     2
Portugal           2
Slovenia           2
Finland            2
France (1963-)     2
Spain              2
Denmark            2
Lithuania          2
Ireland            2
Latvia             2
Greece             2
Estonia            2
Hungary            2
Belgium            2
United Kingdom     2
Poland             2
Slovakia           2
United States      2
Netherlands        2
Australia          5
Iceland            6
Israel             6
Japan             12
Turkey            14
New Zealand       15
Korea, South      24
Canada            26
Chile             26
Mexico            26
dtype: int64


We weed out any country that has more than 2 missing values.

In [11]:
bad_countries = [country for country, value in missing_value_per_country.items() if value > 2]
data = data[~(data.cname.isin(bad_countries))]
print(data.head())

       cname ccodealp  year  gfs_envr   wdi_co2  oecd_soxnox_t1a  \
124  Austria      AUT  1995  2.132733  7.538586           46.929   
125  Austria      AUT  1996  2.256715  7.957359           44.087   
126  Austria      AUT  1997  0.723879  7.881156           40.244   
127  Austria      AUT  1998  0.800405  7.998933           35.638   
128  Austria      AUT  1999  0.884917  7.759483           33.723   

     oecd_soxnox_t1b  wdi_forest  
124          199.917   46.100751  
125          217.706   46.175829  
126          204.490   46.250907  
127          216.869   46.325988  
128          208.297   46.401066  


Now we weed out the 2 bad years.

In [12]:
nan_env = nan_env.drop(columns=bad_countries)
years = data.year.value_counts().index
missing_values_per_year = {year : nan_env.loc[year, :].sum() for year in years}
print(pd.Series(missing_values_per_year).sort_values())

2006     0
2017     0
2016     0
2015     0
2014     0
2013     0
2012     0
2011     0
2010     0
2009     0
2008     0
2007     0
1995     0
2005     0
2004     0
2003     0
2002     0
2001     0
2000     0
1999     0
1998     0
1997     0
1996     0
2018     0
2019    22
2020    26
dtype: int64


They are 2019 and 2020.

In [13]:
bad_years = [year for year, val in missing_values_per_year.items() if val > 0]
data = data[~(data.year.isin(bad_years))]

Now let's look at the missing value report for co2 emissions.

In [14]:
nan_co2 = pd.DataFrame(gen_nan_report(data=data, x='wdi_co2'))
countries = data['cname'].value_counts().index
missing_value_per_country_co2 = {country : nan_co2.loc[:, country].sum() for country in countries}
print(pd.Series(missing_value_per_country_co2).sort_values())

Belgium           2
Finland           2
Germany           2
Denmark           2
Spain             2
Luxembourg        2
Slovakia          2
Estonia           2
Hungary           2
Ireland           2
Austria           2
Lithuania         2
Latvia            2
Poland            2
Portugal          2
Czech Republic    2
Norway            2
Sweden            2
United States     2
Netherlands       2
Greece            2
United Kingdom    2
Switzerland       2
Slovenia          2
France (1963-)    4
Italy             4
dtype: int64


We weed out any country that has more than 2 missing values.

In [15]:
bad_countries_co2 = [country for country, value in missing_value_per_country_co2.items() if value > 2]
data = data[~(data.cname.isin(bad_countries_co2))]
print(data.head())

       cname ccodealp  year  gfs_envr   wdi_co2  oecd_soxnox_t1a  \
124  Austria      AUT  1995  2.132733  7.538586           46.929   
125  Austria      AUT  1996  2.256715  7.957359           44.087   
126  Austria      AUT  1997  0.723879  7.881156           40.244   
127  Austria      AUT  1998  0.800405  7.998933           35.638   
128  Austria      AUT  1999  0.884917  7.759483           33.723   

     oecd_soxnox_t1b  wdi_forest  
124          199.917   46.100751  
125          217.706   46.175829  
126          204.490   46.250907  
127          216.869   46.325988  
128          208.297   46.401066  


Now let's weed out 2 more bad years.

In [16]:
nan_co2 = nan_co2.drop(columns=bad_countries_co2)
years = data.year.value_counts().index
missing_values_per_year_co2 = {year : nan_co2.loc[year, :].sum() for year in years}
print(pd.Series(missing_values_per_year_co2).sort_values())

2005     0
2015     0
2014     0
2013     0
2012     0
2011     0
2010     0
2009     0
2008     0
2007     0
2006     0
1995     0
2004     0
2003     0
2002     0
2001     0
2000     0
1999     0
1998     0
1997     0
1996     0
2016     0
2017    24
2018    24
dtype: int64


They are 2017 and 2018.

In [17]:
bad_years = [year for year, val in missing_values_per_year_co2.items() if val > 0]
data = data[~(data.year.isin(bad_years))]

Note that now it is a completely balanced dataset. Now we GDP growth (annual %), General government final consumption expenditure (% of GDP), and Gross capital formation (% of GDP) data from World Bank Data Bank. Then, we will look at how many countries we have and the year range. Access the following links for further data description.

- GDP growth : https://data.worldbank.org/indicator/NY.GDP.MKTP.KD.ZG
- General government final consumption expenditure (% of GDP) : https://data.worldbank.org/indicator/NE.CON.GOVT.ZS
- Gross capital formation (% of GDP) : https://data.worldbank.org/indicator/NE.GDI.TOTL.ZS
- Industry formation (% of GDP) :https://data.worldbank.org/indicator/NV.IND.TOTL.ZS

In [18]:
growth_rates = {}
gov_exp = {}
gross_capital = {}
industry_share = {}

ccodes = data['ccodealp'].value_counts().index

for j in range(len(ccodes)):
    c = ccodes[j]
    growth = wb.get_series('NY.GDP.MKTP.KD.ZG', country=c, id_or_value='id', simplify_index=True)
    govexp = wb.get_series('NE.CON.GOVT.ZS', country=c, id_or_value='id', simplify_index=True)
    grosscap = wb.get_series('NE.GDI.TOTL.ZS', country=c, id_or_value='id', simplify_index=True)
    indshare = wb.get_series('NV.IND.TOTL.ZS', country=c, id_or_value='id', simplify_index=True)
    growth_rates.update({c : growth})
    gov_exp.update({c : govexp})
    gross_capital.update({c : grosscap})
    industry_share.update({c : indshare})

growth_columns = ['ccodealp', 'year', 'growth_rate']
growth_rates = pd.DataFrame(growth_rates).unstack().reset_index()
growth_rates.columns = growth_columns

gov_exp_columns = ['ccodealp', 'year', 'gov_exp']
gov_exp = pd.DataFrame(gov_exp).unstack().reset_index()
gov_exp.columns = gov_exp_columns

gross_capital_columns = ['ccodealp', 'year', 'gross_capital']
gross_capital = pd.DataFrame(gross_capital).unstack().reset_index()
gross_capital.columns = gross_capital_columns

industry_share_columns = ['ccodealp', 'year', 'industry_share']
industry_share = pd.DataFrame(industry_share).unstack().reset_index()
industry_share.columns = industry_share_columns

wb0 = growth_rates.merge(gov_exp, left_on=['ccodealp', 'year'], right_on=['ccodealp', 'year'])
wb1 = wb0.merge(gross_capital, left_on=['ccodealp', 'year'], right_on=['ccodealp', 'year'])
wb_data = wb1.merge(industry_share, left_on=['ccodealp', 'year'], right_on=['ccodealp', 'year'])

wb_data.year = wb_data.year.astype(int)
wb_data = wb_data[np.logical_and(wb_data['year'] >= 1995, wb_data['year'] <= 2016)]

Finally, we merge World Bank data with our data.

In [19]:
data = data.merge(wb_data, left_on=['ccodealp', 'year'], right_on=['ccodealp', 'year'])

We drop United States from the dataset as the datset shows that it contributed zero percent of government expenditure to environmental protection.

In [20]:
data = data[data['ccodealp'] != 'USA']

The final data has the following countries, and it ranges from 1995-2016.

In [21]:
print([x for x in data.cname.value_counts().index])

['Switzerland', 'Belgium', 'United Kingdom', 'Greece', 'Netherlands', 'Sweden', 'Norway', 'Czech Republic', 'Portugal', 'Poland', 'Slovenia', 'Austria', 'Lithuania', 'Ireland', 'Hungary', 'Estonia', 'Slovakia', 'Luxembourg', 'Spain', 'Denmark', 'Germany', 'Finland', 'Latvia']


Save the dataset:

In [25]:
data.to_csv('/Users/brianyang/Desktop/nyu/Capstone/Capstone-Project/data/capstone_data.csv')